## Merge databases

In [67]:
import json
import sys
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd

In [68]:
# file path
weather_path = '../data/raw/meteotrentino-weather-station-data.json'
twitter_path = '../data/raw/social-pulse-trentino.geojson'

In [69]:
# load the data
with open(weather_path) as json_file:
    data_weather = json.load(json_file)

with open(twitter_path) as json_file:
    data_twitter = json.load(json_file)

# load the grid
grid_path = '../data/raw/trentino-grid.geojson'
df_grid = gpd.read_file(grid_path)

In [70]:
# extract features
twitter_features = gpd.GeoDataFrame(data_twitter['features'])
weather_features = gpd.GeoDataFrame(data_weather['features'])

In [78]:
start_timestamp = 1383260400
list(weather_features['timestamp'] - start_timestamp)
weather_features

,station,geomPoint.geom,elevation,date,timestamp,minTemperature,maxTemperature,precipitation,minWind,maxWind,...,winds.2130,winds.2145,winds.2200,winds.2215,winds.2230,winds.2245,winds.2300,winds.2315,winds.2330,winds.2345
0,T0071,"{'type': 'Point', 'coordinates': [10.79582897,...",905,2013-11-01,1383260400,4.5,12.3,False,0.0,2.5,...,0@183,,0@157,,0.4@228,0.3@248,,0@201,0@199,
1,T0032,"{'type': 'Point', 'coordinates': [11.25371981,...",1155,2013-11-01,1383260400,6.5,10.2,False,NaN,NaN,...,,,,,,,,,,
2,T0096,"{'type': 'Point', 'coordinates': [11.6645808, ...",1205,2013-11-01,1383260400,6.5,11.8,False,NaN,NaN,...,,,,,,,,,,
3,T0074,"{'type': 'Point', 'coordinates': [10.91841055,...",720,2013-11-01,1383260400,6.2,13.6,False,0.0,4.8,...,0.3@198,0.3@196,0.4@236,,0.9@15,0.6@345,1@244,0.5@192,0@157,
4,T0101,"{'type': 'Point', 'coordinates': [11.07973339,...",201,2013-11-01,1383260400,11.1,16.3,False,NaN,NaN,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2190,T0431,"{'type': 'Point', 'coordinates': [11.33626355,...",1055,2013-12-31,1388444400,-5.5,0.5,False,NaN,NaN,...,,,,,,,,,,
2191,T0428,"{'type': 'Point', 'coordinates': [10.64240944,...",525,2013-12-31,1388444400,-2.2,6.9,False,NaN,NaN,...,,,,,,,,,,
2192,T0437,"{'type': 'Point', 'coordinates': [11.76685208,...",1465,2013-12-31,1388444400,-9.9,-2.4,False,0.1,2.5,...,0.6@114,1@113,1.3@105,1.1@115,0.9@113,1@115,1.4@110,1.4@98,1.5@109,1.5@106
2193,T0469,"{'type': 'Point', 'coordinates': [11.6299111, ...",801,2013-12-31,1388444400,-4.1,2.7,False,0.1,1.4,...,0.7@38,0.7@83,0.3@148,,0.8@35,0.5@349,0.7@340,0.4@353,0.8@353,0.7@15


In [72]:
#pd.merge(twitter_features, weather_features, how='inner', on='timestamp')

In [73]:
#len(set(twitter_features['user']))

In [74]:
# remove unnecessary columns
twitter_features = twitter_features.drop('entities', axis=1)
twitter_features = twitter_features.drop('municipality.acheneID', axis=1)
print(twitter_features.columns)

# split date and time 
twitter_features['date'] = twitter_features['created'].str.split('T').str[0]
twitter_features['time'] = twitter_features['created'].str.split('T').str[1]
twitter_features = twitter_features.drop('created', axis=1)
twitter_features = twitter_features[['date', 'time', 'timestamp', 'user', 'geomPoint.geom', 'municipality.name', 'language']]

# keep only minutes
twitter_features['time'] = twitter_features['time'].str.rsplit(':', n=1).str[0]
#print(twitter_features)

Index(['created', 'timestamp', 'user', 'geomPoint.geom', 'municipality.name',
       'language'],
      dtype='object')


In [75]:
# make blocks of 15 minutes (column: hour_blocks)
minutes = twitter_features['time'].str.rsplit(':', n=1).str[1].astype(int)
minutes = (minutes // 15) * 15
minutes = minutes.astype(str)
minutes = minutes.str.zfill(2)
#print(minutes)

hours = twitter_features['time'].str.rsplit(':', n=1).str[0]

twitter_features['hour_blocks'] = hours + minutes
twitter_features

,date,time,timestamp,user,geomPoint.geom,municipality.name,language,hour_blocks
0,2013-11-01,00:04,1383260656,5fd4f31f75,"{'type': 'Point', 'coordinates': [11.13, 46.07]}",Trento,it,0000
1,2013-11-01,00:01,1383260474,68c0e98182,"{'type': 'Point', 'coordinates': [10.83, 46.23]}",Pinzolo,tl,0000
2,2013-11-01,00:43,1383262983,abe21fc052,"{'type': 'Point', 'coordinates': [11.46, 46.29]}",Cavalese,en,0030
3,2013-11-01,00:38,1383262717,94d1efbbfd,"{'type': 'Point', 'coordinates': [11.04, 45.89]}",Rovereto,it,0030
4,2013-11-01,00:45,1383263140,d261d03075,"{'type': 'Point', 'coordinates': [11.12, 46.2]}",San Michele all'Adige,ru,0045
...,...,...,...,...,...,...,...,...
27936,2013-12-31,00:25,1388445948,826558e00e,"{'type': 'Point', 'coordinates': [11.68, 46.42]}",Vigo di Fassa,it,0015
27937,2013-12-29,13:12,1388319166,06e9b1cdff,"{'type': 'Point', 'coordinates': [11.6, 46.31]}",Predazzo,it,1300
27938,2013-11-19,00:51,1384818699,e61ce711d3,"{'type': 'Point', 'coordinates': [11.3, 46.11]}",Sant'Orsola Terme,it,0045
27939,2013-11-23,09:49,1385196576,e61ce711d3,"{'type': 'Point', 'coordinates': [11.3, 46.11]}",Sant'Orsola Terme,it,0945
